Add to cleaning steps
1. Remove `'[deleted]'`
2. Set datatypes for imports
3. Review `flair` columns for refinement or subsetting
4. Clean up `is_video`
5. Process `media` column to extract author name, link, other content for later use

In [116]:
# all_posts_list = []
# for file in os.listdir('./Data/posts/cleaned/'):
#     all_posts_list.append(pd.read_csv(f'./Data/posts/cleaned/{file}'))

# columns_ref = ['subreddit', 'subreddit_id', 'subreddit_type', 'id', 'name', 'media',
#        'is_video', 'created_utc', 'num_comments', 'score', 'ups', 'selftext',
#        'author_flair_text', 'link_flair_text', 'poll_data', 'created']

# all_posts = pd.DataFrame(columns=columns_ref)
# for file in all_posts_list:
#     all_posts = pd.concat([all_posts, file.fillna('')])

deleted_posts = all_posts[(all_posts['selftext']=='[deleted]')|(all_posts['selftext']=='[removed]')].index
# complete_posts = all_posts.drop(index=deleted_posts)

# len(all_posts)

# sentiment_data.to_csv('./Data/posts/cleaned/sentiment_data.csv', index=False)
# sentiment_data.reset_index(inplace=True)

In [120]:
sentiment_data = complete_posts[(complete_posts['media']=='')&(complete_posts['selftext']!='')][['subreddit','id','selftext']]
# sentiment_data = sentiment_data[sentiment_data['media']=='']
print(len(all_posts), len(complete_posts), len(sentiment_data))
sentiment_data['subreddit'].value_counts(normalize=True).round(2)*100

577969 113287 59825


subreddit
Depop          31.0
Etsy           25.0
Flipping       22.0
poshmark        9.0
EtsySellers     8.0
stockx          3.0
Grailed         3.0
Name: proportion, dtype: float64

## Removed deleted & media-only posts, duplicates.
Resulted in 59,825 unique posts.

| site | percent |
|--|--|
| depop | 31% |
| etsy | 25% |
| flipping | 22% |
| poshmark | 9% |
| etsySellers | 8% |
| stockx | 3% |
| grailed | 3% |

In [42]:
import os
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

sentiment_data = pd.read_csv('./Data/posts/cleaned/sentiment_data.csv')

In [111]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')
lemmatizer = WordNetLemmatizer()
all_stopwords = stopwords.words('english') +['anyone','someone','everyone','everybody','also','you','your','etsy','depop','grailed','ebay',
                                             'http','www','com','reddit']
# come back and make feature that notes if the post mentions another site that's not the sub

def post_processor(sentence: str, tokenizer, lemmatizer, stopwords) -> str:
    tokenized = tokenizer.tokenize(sentence.lower())
    lemmatized = [lemmatizer.lemmatize(i) for i in tokenized]
    no_stopwords = [w for w in lemmatized if w not in all_stopwords]
    return ' '.join(no_stopwords)
    
j = 0
total_posts = len(sentiment_data)
cleaned_posts = []
for post in sentiment_data['selftext']:
    cleaned_posts.append(post_processor(post, tokenizer, lemmatizer, all_stopwords))
    
    if j+1 % 10_000 == 0:
        print(j/total_posts)
    
    j+=1

In [112]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    analyzer='word',
    tokenizer=None,
    preprocessor=None,
    stop_words=None,
    max_features=2000
)

In [113]:
output = vectorizer.fit_transform(cleaned_posts)

In [115]:
pd.DataFrame(output.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)[0:20]

item        38865
wa          37722
like        22997
get         21755
seller      21308
would       21227
shipping    18304
know        18270
shop        18007
one         17118
time        17083
ha          16869
sale        16409
amp         16366
buyer       15328
want        14553
day         14464
new         14437
listing     13958
sell        13610
dtype: int64

In [ ]:
posts.dropna(subset='selftext', inplace=True)

In [ ]:
topics = [
    'buyer','customer',
    'item','product',
    'sale','sell','selling','sold','purchase','refund','return'
    'money','paypal','offer','price','free','pay','fee','cost',
    'listing','post','order',
    'flipping',
    'account','review','store','shop','business',
    'photo',
    'shipping','ship','sent','package','shipped','label','tracking','send',
    'received',
    'case', 'issue', # perhaps contested shipments or returns?
    'advice','look','looking','search'
]